In [46]:
import os
import json
import tqdm
from multiprocessing import Pool
import pickle
from math import log2

In [3]:
os.getcwd()

'C:\\Users\\Chankoo\\Desktop\\GitHub\\BOAZ-projects\\airbnb-NLP'

In [6]:
os.chdir('./../airbnb-data')

In [9]:
with open('pos_review.json') as fp:
    review = json.load(fp)

In [11]:
with open('voca_freq_dic.json') as fp:
    voca_freq_dic = json.load(fp)

In [15]:
voca_freq_dic2 = {}
for k,v in voca_freq_dic.items():
    if v>9:
        voca_freq_dic2[k] = v

In [16]:
unq_tokens = list(voca_freq_dic2.keys())

In [17]:
len(unq_tokens)

22384

In [31]:
reviews_tokens = {} # key를 'listingID_reviewIDX'로 갖는 word-freq 딕셔너리
maxCount = {}
for k,rev_lst in review.items():
        for i,rev in enumerate(rev_lst):
            rev_tokens = {}
            maxCount[k+'_'+str(i)] = 0
            for sent in rev[1]:
                for w in sent:
                    if w in rev_tokens:
                        rev_tokens[w] += 1
                    else:
                        rev_tokens[w] = 1
                        
                    if maxCount[k+'_'+str(i)] < rev_tokens[w]:
                        maxCount[k+'_'+str(i)] = rev_tokens[w]
            
            reviews_tokens[k+'_'+str(i)] = rev_tokens

In [32]:
list(reviews_tokens.items())[1]

('10030136_1',
 {'bedroom_N': 1,
  'caught_J': 1,
  'doubledecked_J': 1,
  'fortunately_R': 1,
  'go_V': 1,
  'good_J': 1,
  'modern_J': 1,
  'occupy_V': 1,
  'pretty_R': 1,
  'room_N': 3,
  'see_V': 1,
  'spacious_J': 2,
  'technologicaloverall_N': 1,
  'typhoon_N': 1,
  'windowsthe_N': 1,
  'windowwhen_V': 1})

In [33]:
maxCount['10030136_1']

3

In [38]:
# # token을 k로 token이 등장한 doc의 리스트를 val로 갖는 invertedIdx 만들자
# 시간이 좀 걸린다

invertedIdx = {}
for token in tqdm.tqdm(unq_tokens):
    invertedIdx[token] = []
    for k,v in reviews_tokens.items():
        if token in v:
            invertedIdx[token].append(k)

100%|████████████████████████████████████████████████████████████████████████████| 22384/22384 [42:55<00:00,  8.69it/s]


In [41]:
with open('invertedIdx.dic','wb') as file: 
    pickle.dump(invertedIdx,file)

In [42]:
with open('invertedIdx.dic','rb') as file: 
    invertedIdx = pickle.load(file)

In [48]:
K = 0.5 # K + (1-K)*(f(t,d)/maxf(t,d)) 최솟값 0.5, 최댓값 1 
TF = {}
for k,v in reviews_tokens.items():
    tfList = {}
    
    for w in v:
        if w not in voca_freq_dic2:
            continue
        tfList[w] =  K + (1-K)*(v[w]/maxCount[k])
#         print('{0} | {1} + {2} *({3}/{4})  =  {5}'.format(
#             w, K, (1-K), v[w], maxCount[k], tfList[w]
#         ))
        
    TF[k] = tfList

In [49]:
docSize = len(reviews_tokens)
TFIDF = {}
"""
TF = doc1|{단어1:0.54, 단어2:056 ...} 
"""
for (k,v) in TF.items():
    idfList = {}
    for t in v:
        idf = log2(docSize/len(invertedIdx[t]))
        idfList[t] = v[t] * idf
#         print('{0} | {1} * log({2} / {3}) = {4}'.format(
#             t, v[t], docSize, len(invertedIdx[t]), idfList[t]
#         ))
    
    TFIDF[k] = idfList

In [57]:
list(TFIDF.items())[1]

('10030136_1',
 {'bedroom_N': 4.493337722366502,
  'caught_J': 8.562412128595014,
  'doubledecked_J': 8.970365160809298,
  'fortunately_R': 7.005007441890266,
  'go_V': 1.9353974236098639,
  'good_J': 1.222203961561637,
  'modern_J': 4.589506784926579,
  'occupy_V': 7.810575097129433,
  'pretty_R': 3.2796436724419538,
  'room_N': 2.346739763742937,
  'see_V': 3.156772180231429,
  'spacious_J': 4.031257166144196,
  'typhoon_N': 6.641254049674895,
  'windowsthe_N': 10.574360067259857})

In [55]:
with open('TFIDF_22384.dic','wb') as file: #TFIDF 결과를 저장
    pickle.dump(TFIDF, file)